## This notebook is aimed to generate the dictionary for comparison of networks between runs. As such it is semi active, as for a finished project this shouldnt be needed

#### similar structure to most of my notebooks - first cell imports various modules, sets variable name and has a list of subjejcts and runs, next cell loads in graph_dict which is used to construct comp_dict, the main loop if very rough both in style and what it contains, may be edited a lot when more direction is decided

In [45]:
# this cell imports the necessary packages - sets variables to point to string keys - sets the sub case and control
# dictionaries as well as creating lists of global node and othe metrics

# this cell imports the necessary packages - sets variables to point to string keys - sets the sub case and control
# dictionaries as well as creating lists of global node and othe metrics


import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle

# set variables that will point to dictionary keys - with the type/output will be in the dict, note these 
# are not based on the output of the analagous networkx function but how I have written into dict


#data/matrices
graph = "graph" #networkx graph basically dictionaries all the way down
data = "data" #pandas data frame 
bin_mat = "binary_matrix" #numpy array 
bin_df = "binary_df"
data_numpy = "data_as_numpy_array" #numpy array
norm_lap = "normalised_laplacian" #numpy array?

#global metrics 
gl_eff = "global_efficiency" #float
loc_eff = "local_efficiency" #float
clus_coef = "clusetering_coefficent" #float
ave_path_length = "ave_path_length" #float
clique_num = "clique_number" #int (poss float but always integer valued)
ass_coef = "assortativity_coefficient" #float
transitivity = "transitivity" #float
rich_club_coeff = "rich_club_coefficient" #currently not working would be float
sw_sigma = "" #currently not working would be float
sw_gamma = "" #currently not working would be float
ave_strength = "average_node_strength" #float
no_edges = "number_of_edges" #int
ave_deg = "average_degree" #int

#individual node/edge metrics and dicts
deg = "degree" #dictionary - nodes as key degree as values 
bet_cent = "betweenness_centrality" # dictionary - nodes as key betweeness as values 
edge_bet_cent = "edge_betweenness_centrality" # dictionary - edge tuple as key edgebetweeness centrality as values 
eigen_cent = "eigenvector_centrality" # dictionary - nodes as eigenvalue centrality as values 
deg_cent = "degree_centrality" # dictionary - nodes as key degree centrality as values 
strength = "node_strength"  # dictionary - nodes as key strength as values 

#other
deg_hist = "degree_histogram" #list - index is degree value is how many nodes with that degree 
norm_lap_spec = "normalised_laplacian_spectrum" #list of floats?


#subject dictionary with a list of runs - corresponds to how connectome file is saved for now hand written 
sub_dict = {"NENAH004" : ["orig", "run2"], "NENAH010" :  ["orig", "run2"], "NENAH011" :  ["orig", "run2"],
                "NENAH012" : ["orig", "run2"], "NENAH015" : ["orig", "run2"], "NENAH016" : ["orig", "run2"], 
                "NENAH022" : ["orig", "run2"], "NENAH024" : ["orig", "run2"], "NENAHC002" : ["orig", "run2", "run3"], 
                "NENAHC003" : ["orig", "run2","run3"], "NENAHC010" : ["orig", "run2", "run3"],
                "NENAHC013" : ["orig", "run2", "run3"], "NENAHC016" : ["orig", "run2", "run3"],
                "NENAHC018" : ["orig", "run2", "run3"], "NENAHC020": ["orig", "run2", "run3"],
                "NENAHC025" : ["orig", "run2"]
           }

case_dict = {"NENAH004" : ["orig", "run2"], "NENAH010" :  ["orig", "run2"], "NENAH011" :  ["orig", "run2"],
                "NENAH012" : ["orig", "run2"], "NENAH015" : ["orig", "run2"], "NENAH016" : ["orig", "run2"], 
                "NENAH022" : ["orig", "run2"], "NENAH024" : ["orig", "run2"]
            }

control_dict = { "NENAHC002" : ["orig", "run2", "run3"],  "NENAHC003" : ["orig", "run2", "run3"], 
                "NENAHC010" : ["orig", "run2", "run3"],"NENAHC013" : ["orig", "run2", "run3"],
                "NENAHC016" : ["orig", "run2", "run3"], "NENAHC018" : ["orig", "run2", "run3"],
                "NENAHC020": ["orig", "run2", "run3"], "NENAHC025" : ["orig", "run2",]
             }

#create the lists which might be usuefull in automating data frame creation
#note these should be strings
global_metrics = [gl_eff, loc_eff, clus_coef, ave_path_length, clique_num,
                  transitivity, ave_strength, no_edges, ave_deg] 

node_metrics = [deg, strength, bet_cent, eigen_cent, deg_cent]

other_metrics = [deg_hist, norm_lap_spec, edge_bet_cent,]



Open main graph dict which is used as data source

In [46]:
with open('graph_dict.pickle', 'rb') as handle:
     graph_dict = pickle.load(handle)

the main loop that generates the comp_dict, the idea is that each subject has a dictionary with various keys and vlaues - often one for each run, but lots caluclated as differences - so far this is not taking into account 3rd runs - will look to do something more statistically significant with them, - one thing to noe is that bin_diff is a binary matrix (technically pandas data frame) taking vallue of one wherever the corresponding edge is present in exactly one run, and 0 elsewhere - hence 'multiplying elementwise (how dataframes natural multiplication is defined where appropriate)' either runs original matrix by it will give a matrix of only the unique edges in that run, this is used at multiple thresholds, 

I may heavily reduce the number of items stored in the comp_dict as many are simple caluclations based off of others
    

In [139]:
# the main generator of the dictionary used to comapre across runs this is very rough and could be cleaned up when I 
# have a better idea at what I want to compare, for now 

# note - need to go through and check no double counting is happening, i believe it is for various node 
# strength categories

comp_dict = {}
thresholds = [0.05, 0.02, 0.01, 0.005, 0.001, 0.0005, 0.0001]
for sub in sub_dict:
    comp_dict[sub] = {}
    comp_dict[sub]['orig_data'] = graph_dict[f'{sub}orig']["data"]
    comp_dict[sub]['run2_data'] = graph_dict[f'{sub}run2']["data"]
    comp_dict[sub]['bin_diff']  = abs(graph_dict[f'{sub}orig']["binary_df_0"] - graph_dict[f'{sub}run2']["binary_df_0"])
    comp_dict[sub]['edges_different'] = comp_dict[sub]['bin_diff'].sum().sum() / 2
    comp_dict[sub]['orig_edges'] = graph_dict[f'{sub}orig']["number_of_edges"]
    comp_dict[sub]['run2_edges'] = graph_dict[f'{sub}run2']["number_of_edges"]
    comp_dict[sub]['average_edges'] = 0.5 * (comp_dict[sub]['orig_edges'] + comp_dict[sub]['run2_edges'])
    comp_dict[sub]['percentage_edge_diff'] = 100* (comp_dict[sub]['edges_different'] / comp_dict[sub]['average_edges'])
    comp_dict[sub]['res_orig_df']= comp_dict[sub]['bin_diff'] * comp_dict[sub]['orig_data']
    comp_dict[sub]['res_orig_bin_df'] = (comp_dict[sub]['res_orig_df']> 0).astype(np.int_)
    comp_dict[sub]['res_run2_df'] = comp_dict[sub]['bin_diff'] * comp_dict[sub]['run2_data']
    comp_dict[sub]['res_run2_bin_df'] = (comp_dict[sub]['res_run2_df'] > 0).astype(np.int_)
    comp_dict[sub]['orig_unique_edges'] = 0.5 * (comp_dict[sub]['res_orig_bin_df'].sum().sum())
    comp_dict[sub]['run2_unique_edges'] = 0.5 * (comp_dict[sub]['res_run2_bin_df'].sum().sum())
    comp_dict[sub]['orig_unique_edges_pct'] = 100 * (comp_dict[sub]['orig_unique_edges'] / comp_dict[sub]['orig_edges'])
    comp_dict[sub]['run2_unique_edges_pct'] = 100 * (comp_dict[sub]['run2_unique_edges'] / comp_dict[sub]['run2_edges'])
    comp_dict[sub]['orig_unique_node_strength'] = comp_dict[sub]['res_orig_df'].sum()
    comp_dict[sub]['orig_unique_node_strength_pct'] = 100 * (comp_dict[sub]['orig_unique_node_strength'] / comp_dict[sub]['orig_data'].sum())
    comp_dict[sub]['run2_unique_node_strength'] = comp_dict[sub]['res_run2_df'].sum()
    comp_dict[sub]['run2_unique_node_strength_pct'] = 100 * (comp_dict[sub]['run2_unique_node_strength'] / comp_dict[sub]['run2_data'].sum())
    comp_dict[sub]['orig_unique_tot_strength'] = comp_dict[sub]['res_orig_df'].sum().sum()
    comp_dict[sub]['orig_unique_tot_strength_pct'] = 100 * (comp_dict[sub]['orig_unique_tot_strength'] / comp_dict[sub]['orig_data'].sum().sum())
    comp_dict[sub]['run2_unique_tot_strength'] = comp_dict[sub]['res_run2_df'].sum().sum()
    comp_dict[sub]['run2_unique_tot_strength_pct'] = 100 * (comp_dict[sub]['run2_unique_tot_strength'] / comp_dict[sub]['run2_data'].sum().sum())
    for thresh in thresholds:
        comp_dict[sub][f'unique_orig_{thresh}_bin'] = (comp_dict[sub]['res_orig_df']> thresh).astype(np.int_)
        comp_dict[sub][f'unique_orig_{thresh}'] = comp_dict[sub]['orig_data'] * comp_dict[sub][f'unique_orig_{thresh}_bin']
        comp_dict[sub][f'unique_run2_{thresh}_bin' ]= (comp_dict[sub]['res_run2_df'] > thresh).astype(np.int_)
        comp_dict[sub][f'unique_run2_{thresh}' ]= comp_dict[sub]['run2_data']* comp_dict[sub][f'unique_run2_{thresh}_bin' ]
        comp_dict[sub][f'no_edges_unique_orig_{thresh}']= 0.5 * (comp_dict[sub][f'unique_orig_{thresh}_bin'].sum().sum())
        comp_dict[sub][f'no_edges_unique_run2_{thresh}' ]= 0.5 * (comp_dict[sub][f'unique_run2_{thresh}_bin'].sum().sum())
        comp_dict[sub][f'percentage_unique_orig_{thresh}'] = comp_dict[sub][f'no_edges_unique_orig_{thresh}'] / comp_dict[sub]['orig_edges']
        comp_dict[sub][f'percentage_unique_run2_{thresh}'] = comp_dict[sub][f'no_edges_unique_run2_{thresh}'] / comp_dict[sub]['run2_edges']
        comp_dict[sub][f'diff_by_{thresh}'] = (abs(graph_dict[f'{sub}orig'][data] - graph_dict[f'{sub}run2'][data]) > thresh).astype(np.int_)

In [140]:
print(comp_dict.keys())

dict_keys(['NENAH004', 'NENAH010', 'NENAH011', 'NENAH012', 'NENAH015', 'NENAH016', 'NENAH022', 'NENAH024', 'NENAHC002', 'NENAHC003', 'NENAHC010', 'NENAHC013', 'NENAHC016', 'NENAHC018', 'NENAHC020', 'NENAHC025'])


In [141]:
comp_dict['NENAHC025']['bin_diff'].sum().sum()

898

In [142]:
comp_dict['NENAHC025']['unique_run2_0.005_bin'].sum().sum()

2

In [ ]:
for sub in sub_dict :
    print(sub)
    print(comp_dict[sub]['percentage_edge_diff'])

In [ ]:
for sub in sub_dict :
    print(sub)
    print(comp_dict[sub]['orig_unique_tot_strength_pct'])
    print(comp_dict[sub]['run2_unique_tot_strength_pct'])

The next two cells are for saving and loading (via pickle) and will use space below for testing 

In [146]:
with open('comp_dict.pickle', 'wb') as handle:
    pickle.dump(comp_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('comp_dict.pickle', 'rb') as handle:
    comp_dict = pickle.load(handle)